<a href="https://colab.research.google.com/github/Lee-Minsoo-97/Decision-Modeling/blob/main/Costco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gurobipy

In [ ]:
from gurobipy import Model, GRB
import pandas as pd
import numpy as np

# Define data for destinations, distances, and stops
destination_names = [
    "Atlanta", "Everett", "Ephrata", "Riverview", "Carson",
    "Chamblee", "Roseville", "Hanover", "Sparks", "Parsippany",
    "Effingham", "Kearny"
]
distances = [612, 612, 190, 383, 3063, 429, 600, 136, 2439, 355, 570, 324]  # Distance in miles
num_stops = [0, 3, 0, 0, 2, 0, 3, 0, 0, 1, 0, 0]  # Number of intermediate stops for each destination

# Carrier rates (in dollars per mile) for each destination (NaN means carrier does not serve destination)
carrier_rates = {
    'A': [np.nan, np.nan, np.nan, 0.79, np.nan, np.nan, 1.24, np.nan, np.nan, np.nan, 0.87, np.nan],
    'B': [0.88, 1.18, 3.42, 1.01, 0.80, 1.23, 1.13, 4.78, 1.45, 1.62, 0.87, 2.01],
    'C': [1.15, 1.27, 1.73, 1.25, 0.87, 1.61, 1.89, 2.23, np.nan, 1.36, 1.25, 1.54],
    'D': [0.87, 1.39, 1.71, 0.96, np.nan, 1.22, 1.32, 2.39, 1.20, 1.39, 0.87, 1.53],
    'E': [0.95, 1.35, 1.82, 0.95, 1.00, 1.33, 1.41, 2.26, np.nan, 1.03, 0.90, 1.28],
    'F': [1.05, 1.28, 2.00, 1.11, np.nan, 1.47, 1.41, 2.57, np.nan, 1.76, 1.31, 1.95]
}

# Fixed costs and charges
minimum_cost_per_load = {'A': 350, 'B': 400, 'C': 350, 'D': 300, 'E': 350, 'F': 300}
stop_off_charge = {'A': 50, 'B': 75, 'C': 50, 'D': 35, 'E': 50, 'F': 50}


In [ ]:
# Step 1: Calculate cost matrix dynamically
cost_matrix = []
for i in range(len(destination_names)):
    row = []
    for carrier in carrier_rates.keys():
        if not np.isnan(carrier_rates[carrier][i]):
            # Calculate cost as mileage cost + stop-off cost
            mileage_cost = distances[i] * carrier_rates[carrier][i]
            stop_cost = num_stops[i] * stop_off_charge[carrier]
            total_cost = max(mileage_cost + stop_cost, minimum_cost_per_load[carrier])
        else:
            total_cost = np.inf  # Set as infinite if carrier cannot serve the destination
        row.append(total_cost)
    cost_matrix.append(row)

# Convert cost matrix into a DataFrame for better readability
cost_df = pd.DataFrame(cost_matrix, columns=carrier_rates.keys(), index=destination_names)

In [ ]:
# Step 2: Define requirements and constraints
# Number of trips required for each destination
trips_required = [4, 1, 3, 5, 1, 1, 1, 1, 2, 1, 5, 7]

# Maximum number of trips each carrier can handle
carrier_capacity = {
    'A': 4,
    'B': 8,
    'C': 7,
    'D': 7,
    'E': 3,
    'F': 4
}

# Minimum number of trips each carrier must handle
minimum_carrier_commitment = {
    'A': 1,
    'B': 7,
    'C': 6,
    'D': 0,
    'E': 0,
    'F': 4
}

In [ ]:
# Step 3: Initialize the optimization model
model = Model("Costco_Distribution_Optimization")

# Decision variables: x[destination, carrier] represents trips assigned
x = model.addVars(len(destination_names), len(carrier_rates), vtype=GRB.INTEGER, name="x")

In [ ]:
# Step 4: Define the objective function (minimize total cost)
model.setObjective(
    sum(
        x[i, j] * (cost_df.iloc[i, j] if np.isfinite(cost_df.iloc[i, j]) else 1e6)
        for i in range(len(destination_names))
        for j in range(len(carrier_rates))
    ),
    GRB.MINIMIZE
)


In [ ]:
# Step 5: Add constraints
# 5.1: Ensure all trips are assigned for each destination
for i in range(len(destination_names)):
    model.addConstr(
        sum(x[i, j] for j in range(len(carrier_rates))) == trips_required[i],
        name=f"Trips_Assigned_{destination_names[i]}"
    )

# 5.2: Carrier capacity constraints
for j, carrier in enumerate(carrier_rates.keys()):
    model.addConstr(
        sum(x[i, j] for i in range(len(destination_names))) <= carrier_capacity[carrier],
        name=f"Carrier_Capacity_{carrier}"
    )

# 5.3: Minimum carrier commitment constraints
for j, carrier in enumerate(carrier_rates.keys()):
    model.addConstr(
        sum(x[i, j] for i in range(len(destination_names))) >= minimum_carrier_commitment[carrier],
        name=f"Carrier_Commitment_{carrier}"
    )


In [ ]:
# Step 6: Solve the model
model.optimize()

# Check if the solution is optimal
if model.status == GRB.OPTIMAL:
    # Extract the assignment matrix and round to integers
    optimal_assignment = pd.DataFrame(
        [[round(x[i, j].x) for j in range(len(carrier_rates))] for i in range(len(destination_names))],
        columns=carrier_rates.keys(),
        index=destination_names
    )
    # Print the table
    print("Optimal Carrier Assignment:")
    print(optimal_assignment.to_string(float_format=lambda x: f"{int(x):d}"))
    print(f"\nTotal Cost: ${model.objVal:.2f}")
else:
    print("Optimization was not successful. Status:", model.status)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 24 rows, 72 columns and 216 nonzeros
Model fingerprint: 0x52fd94ac
Variable types: 0 continuous, 72 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 1e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+00]
Presolved: 21 rows, 72 columns, 180 nonzeros

Continuing optimization...


Explored 1 nodes (22 simplex iterations) in 0.09 seconds (0.00 work units)
Thread count was 2 (of 2 available processors)

Solution count 2: 22394.4 4.0221e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.239438000000e+04, best bound 2.239438000000e+04, gap 0.0000%
Optimal Carrier Assignment:
            A  B  C  D  E  F
Atlanta     0  0  0  4  0  0
Everett     0  0  0